In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00


In [14]:
from google.colab import files

uploaded_files = files.upload()


Saving training1.py to training1.py
Saving training2.py to training2.py


In [15]:
!ls /content/

app.py	drive  sample_data  training1.py  training2.py


In [3]:
%%writefile app.py
import gradio as gr

def detect_safety(video):
    return "Processing Video for Women's Safety Analysis..."

iface = gr.Interface(
    fn=detect_safety,
    inputs=gr.Video(),
    outputs="text",
    title="Women's Safety AI System",
    description="Upload a video to analyze potential threats and ensure safety."
)

iface.launch(share=True)


Writing app.py


In [20]:
%%writefile app.py
import gradio as gr
import torch
import cv2
import numpy as np
import sys
import os

# Ensure training1.py and training2.py are in the correct path
sys.path.append("/content/")

try:
    from training1 import GenderClassificationModel, ViolenceDetectionModel  # Import models from training1.py
    from training2 import CrowdDetectionModel  # Import model from training2.py
except ModuleNotFoundError as e:
    print("❌ Error: training1.py or training2.py not found. Make sure they are uploaded in /content/")
    raise e

# ✅ Load the Gender Classification Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gender_model = GenderClassificationModel().to(device)

if os.path.exists("gender_classification.pth"):
    gender_model.load_state_dict(torch.load("gender_classification.pth", map_location=device))
    gender_model.eval()
else:
    print("❌ Error: gender_classification.pth not found!")

# ✅ Load the Violence Detection Model
violence_model = ViolenceDetectionModel().to(device)
if os.path.exists("violence_detection_model.pth"):
    violence_model.load_state_dict(torch.load("violence_detection_model.pth", map_location=device))
    violence_model.eval()
else:
    print("❌ Error: violence_detection_model.pth not found!")

# ✅ Load the Crowd Detection Model
crowd_model = CrowdDetectionModel().to(device)
if os.path.exists("crowd_detection.pth"):
    crowd_model.load_state_dict(torch.load("crowd_detection.pth", map_location=device))
    crowd_model.eval()
else:
    print("❌ Error: crowd_detection.pth not found!")

# ✅ Function to process video frames
def detect_safety(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return "Error: No frames found in the video."

    # ✅ Run Gender Classification on the first frame
    frame = cv2.resize(frames[0], (128, 128))
    frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
    gender_pred = torch.argmax(gender_model(frame_tensor)).item()
    gender_result = "Female" if gender_pred == 1 else "Male"

    # ✅ Run Violence Detection
    violence_pred = torch.argmax(violence_model(frame_tensor)).item()
    violence_result = "Violence Detected" if violence_pred == 1 else "No Violence"

    # ✅ Run Crowd Detection
    frame = cv2.resize(frames[0], (224, 224))
    frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
    crowd_pred = torch.argmax(crowd_model(frame_tensor)).item()
    crowd_result = "Crowd Present" if crowd_pred == 1 else "No Crowd"

    # ✅ Decision Logic
    alert_message = "✅ Safe Environment"
    if gender_result == "Female" and (violence_pred == 1 or crowd_pred == 1):
        alert_message = "🚨 Emergency! Threat Detected!"

    return f"""
    Gender: {gender_result}
    Violence: {violence_result}
    Crowd Detection: {crowd_result}
    Alert: {alert_message}
    """

# ✅ Gradio Interface
iface = gr.Interface(
    fn=detect_safety,
    inputs=gr.Video(),
    outputs="text",
    title="👩‍🦰 Women's Safety AI System",
    description="Upload a video to analyze potential threats and ensure safety."
)

iface.launch(share=True)


Overwriting app.py


In [21]:
!python app.py


Path to dataset files: /root/.cache/kagglehub/datasets/mohamedmustafa/real-life-violence-situations-dataset/versions/1
Subfolders: ['real life violence situations', 'Real Life Violence Dataset']
Contents of 'real life violence situations': ['Real Life Violence Dataset']
Contents of 'Real Life Violence Dataset': ['Violence', 'NonViolence']
real life violence situations contains 1 files.
Real Life Violence Dataset contains 2 files.
Subfolders: ['Violence', 'NonViolence']
Violence: ['V_614.mp4', 'V_415.mp4', 'V_87.mp4', 'V_659.mp4', 'V_504.mp4', 'V_740.mp4', 'V_3.mp4', 'V_351.mp4', 'V_730.mp4', 'V_600.mp4']
NonViolence: ['NV_877.avi', 'NV_638.mp4', 'NV_430.mp4', 'NV_926.mp4', 'NV_685.mp4', 'NV_543.mp4', 'NV_719.mp4', 'NV_301.mp4', 'NV_275.mp4', 'NV_908.avi']
2025-03-21 15:08:09.108023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742569689.1

In [22]:
%%writefile /content/training1.py
import torch  # ✅ Add missing import
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Dense, Flatten, TimeDistributed, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# ✅ Ensure model training and saving does not cause issues
if __name__ == "__main__":
    model = nn.Linear(10, 2)  # Dummy model to avoid errors (Replace with actual model)
    model_path = "/content/violence_detection_model.pth"
    torch.save(model.state_dict(), model_path)
    print(f"✅ Model saved at {model_path}")


Overwriting /content/training1.py


In [23]:
!python /content/training1.py


2025-03-21 15:33:33.600378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742571213.621781  125687 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742571213.628386  125687 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:33:33.650027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
✅ Model saved at /content/violence_detection_model.pth


In [24]:
!python app.py


2025-03-21 15:34:00.391322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742571240.411896  125820 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742571240.419833  125820 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:34:00.441678: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/app.py", line 12, in <module>
    from training1 import GenderClassificat

In [26]:
!cat /content/training1.py | grep "class"


In [27]:
%%writefile app.py
import gradio as gr
import torch
import cv2
import numpy as np
import sys
import os


sys.path.append("/content/")

try:
    import training1
    import training2
except ModuleNotFoundError as e:
    print("❌ Error: training1.py or training2.py not found. Make sure they are uploaded in /content/")
    raise e


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if hasattr(training1, "GenderClassificationModel"):
    gender_model = training1.GenderClassificationModel().to(device)
    if os.path.exists("gender_classification.pth"):
        gender_model.load_state_dict(torch.load("gender_classification.pth", map_location=device))
        gender_model.eval()
    else:
        print("❌ Error: gender_classification.pth not found!")
else:
    print("⚠ Warning: GenderClassificationModel not found in training1.py")
    gender_model = None

if hasattr(training1, "ViolenceDetectionModel"):
    violence_model = training1.ViolenceDetectionModel().to(device)
    if os.path.exists("violence_detection_model.pth"):
        violence_model.load_state_dict(torch.load("violence_detection_model.pth", map_location=device))
        violence_model.eval()
    else:
        print("❌ Error: violence_detection_model.pth not found!")
else:
    print("⚠ Warning: ViolenceDetectionModel not found in training1.py")
    violence_model = None

if hasattr(training2, "CrowdDetectionModel"):
    crowd_model = training2.CrowdDetectionModel().to(device)
    if os.path.exists("crowd_detection.pth"):
        crowd_model.load_state_dict(torch.load("crowd_detection.pth", map_location=device))
        crowd_model.eval()
    else:
        print("❌ Error: crowd_detection.pth not found!")
else:
    print("⚠ Warning: CrowdDetectionModel not found in training2.py")
    crowd_model = None


def detect_safety(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return "Error: No frames found in the video."

    results = []


    if gender_model:
        frame = cv2.resize(frames[0], (128, 128))
        frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        gender_pred = torch.argmax(gender_model(frame_tensor)).item()
        gender_result = "Female" if gender_pred == 1 else "Male"
        results.append(f"Gender: {gender_result}")


    if violence_model:
        violence_pred = torch.argmax(violence_model(frame_tensor)).item()
        violence_result = "Violence Detected" if violence_pred == 1 else "No Violence"
        results.append(f"Violence: {violence_result}")


    if crowd_model:
        frame = cv2.resize(frames[0], (224, 224))
        frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        crowd_pred = torch.argmax(crowd_model(frame_tensor)).item()
        crowd_result = "Crowd Present" if crowd_pred == 1 else "No Crowd"
        results.append(f"Crowd Detection: {crowd_result}")

    alert_message = "✅ Safe Environment"
    if "Female" in results and ("Violence Detected" in results or "Crowd Present" in results):
        alert_message = "🚨 Emergency! Threat Detected!"

    return "\n".join(results) + f"\nAlert: {alert_message}"


iface = gr.Interface(
    fn=detect_safety,
    inputs=gr.Video(),
    outputs="text",
    title="👩‍🦰 Women's Safety AI System",
    description="Upload a video to analyze potential threats and ensure safety."
)

iface.launch(share=True)


Overwriting app.py


In [28]:
!python app.py


2025-03-21 15:35:58.998032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742571359.019179  126342 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742571359.025357  126342 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:35:59.048115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/app.py", line 13, in <module>
    import training2
  File "/content/train

In [29]:
%%writefile /content/training2.py
import torch  # Ensure PyTorch is imported
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Define any necessary classes or models
class CrowdDetectionModel(tf.keras.Model):
    def __init__(self):
        super(CrowdDetectionModel, self).__init__()
        self.model = tf.keras.applications.MobileNetV2(weights=None, input_shape=(224, 224, 3), classes=2)

    def call(self, inputs):
        return self.model(inputs)

# ✅ Instantiate the model
crowd_model = CrowdDetectionModel()


Overwriting /content/training2.py


In [30]:
!python /content/training2.py


2025-03-21 15:37:13.674448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742571433.694856  126692 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742571433.701208  126692 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:37:13.722077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 15:37:18.540809: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting becau

In [31]:
!python app.py


2025-03-21 15:37:44.356582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742571464.376981  126834 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742571464.383184  126834 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:37:44.404266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 15:37:47.703567: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting becau

In [32]:
%%writefile app.py
import gradio as gr
import torch
import cv2
import numpy as np
import sys
import os
import tensorflow as tf

# Ensure training1.py and training2.py are in the correct path
sys.path.append("/content/")

try:
    import training1
    import training2
except ModuleNotFoundError as e:
    print("Error: training1.py or training2.py not found. Make sure they are uploaded in /content/")
    raise e

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


gender_model = None
violence_model = None
if hasattr(training1, "GenderClassificationModel"):
    gender_model = training1.GenderClassificationModel().to(device)
    if os.path.exists("gender_classification.pth"):
        gender_model.load_state_dict(torch.load("gender_classification.pth", map_location=device))
        gender_model.eval()
    else:
        print("❌ Error: gender_classification.pth not found!")
else:
    print("⚠ Warning: GenderClassificationModel not found in training1.py")

if hasattr(training1, "ViolenceDetectionModel"):
    violence_model = training1.ViolenceDetectionModel().to(device)
    if os.path.exists("violence_detection_model.pth"):
        violence_model.load_state_dict(torch.load("violence_detection_model.pth", map_location=device))
        violence_model.eval()
    else:
        print("❌ Error: violence_detection_model.pth not found!")
else:
    print("⚠ Warning: ViolenceDetectionModel not found in training1.py")


crowd_model = None
if hasattr(training2, "CrowdDetectionModel"):
    crowd_model = training2.CrowdDetectionModel()
    if os.path.exists("crowd_detection.h5"):
        crowd_model.load_weights("crowd_detection.h5")
    else:
        print("❌ Error: crowd_detection.h5 not found!")
else:
    print("⚠ Warning: CrowdDetectionModel not found in training2.py")

def detect_safety(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return "Error: No frames found in the video."

    results = []


    if gender_model:
        frame = cv2.resize(frames[0], (128, 128))
        frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        gender_pred = torch.argmax(gender_model(frame_tensor)).item()
        gender_result = "Female" if gender_pred == 1 else "Male"
        results.append(f"Gender: {gender_result}")

    if violence_model:
        violence_pred = torch.argmax(violence_model(frame_tensor)).item()
        violence_result = "Violence Detected" if violence_pred == 1 else "No Violence"
        results.append(f"Violence: {violence_result}")


    if crowd_model:
        frame = cv2.resize(frames[0], (224, 224))
        frame_tensor = np.expand_dims(frame, axis=0) / 255.0  # Normalize for TensorFlow
        crowd_pred = np.argmax(crowd_model.predict(frame_tensor))
        crowd_result = "Crowd Present" if crowd_pred == 1 else "No Crowd"
        results.append(f"Crowd Detection: {crowd_result}")


    alert_message = "✅ Safe Environment"
    if "Female" in results and ("Violence Detected" in results or "Crowd Present" in results):
        alert_message = "🚨 Emergency! Threat Detected!"

    return "\n".join(results) + f"\nAlert: {alert_message}"


iface = gr.Interface(
    fn=detect_safety,
    inputs=gr.Video(),
    outputs="text",
    title="👩‍🦰 Women's Safety AI System",
    description="Upload a video to analyze potential threats and ensure safety."
)

iface.launch(share=True)


Overwriting app.py


In [33]:
!python app.py


2025-03-21 15:39:02.539920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742571542.560337  127215 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742571542.566817  127215 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:39:02.588386: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 15:39:07.544290: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting becau

In [34]:
!ls /content/


app.py	drive  __pycache__  sample_data  training1.py  training2.py  violence_detection_model.pth


In [35]:
from google.colab import files
uploaded = files.upload()


Saving training2.py to training2 (1).py
Saving training1.py to training1 (1).py


In [36]:
!ls /content/

 app.py   __pycache__  'training1 (1).py'  'training2 (1).py'   violence_detection_model.pth
 drive	  sample_data   training1.py	    training2.py


In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np


model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")  # Binary classification (Crowd / No Crowd)
])

# ✅ Step 2: Compile the Model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# ✅ Step 3: Create Dummy Training Data (Since We Have No Dataset)
X_train = np.random.rand(100, 224, 224, 3)  # 100 random images
y_train = np.random.randint(0, 2, 100)  # Random labels (0 = No Crowd, 1 = Crowd)

# ✅ Step 4: Train the Model
model.fit(X_train, y_train, epochs=5, batch_size=10)

# ✅ Step 5: Save the Model as `crowd_detection.h5`
model.save("crowd_detection.h5")
print("✅ Model saved as crowd_detection.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5361 - loss: 7.8077
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4773 - loss: 0.7098
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6467 - loss: 0.6906
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6230 - loss: 0.6799
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6480 - loss: 0.6217


✅ Model saved as crowd_detection.h5


In [38]:
!ls /content/


 app.py		      __pycache__	  training1.py	      violence_detection_model.pth
 crowd_detection.h5   sample_data	 'training2 (1).py'
 drive		     'training1 (1).py'   training2.py


In [39]:
!python app.py


2025-03-21 15:45:15.203018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742571915.222979  128951 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742571915.229267  128951 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:45:19.849642: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1742571919.849849  128951 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12798 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


In [44]:
%%writefile app.py
import gradio as gr
import torch
import cv2
import numpy as np
import sys
import os
import tensorflow as tf
from cryptography.fernet import Fernet


sys.path.append("/content/")

try:
    import training1
    import training2
except ModuleNotFoundError as e:
    print("Error: training1.py or training2.py not found. Make sure they are uploaded in /content/")
    raise e


key = Fernet.generate_key()
cipher = Fernet(key)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


gender_model = None
violence_model = None
if hasattr(training1, "GenderClassificationModel"):
    gender_model = training1.GenderClassificationModel().to(device)
    if os.path.exists("gender_classification.pth"):
        gender_model.load_state_dict(torch.load("gender_classification.pth", map_location=device))
        gender_model.eval()
    else:
        print("Error: gender_classification.pth not found!")
else:
    print("⚠ Warning: GenderClassificationModel not found in training1.py")

if hasattr(training1, "ViolenceDetectionModel"):
    violence_model = training1.ViolenceDetectionModel().to(device)
    if os.path.exists("violence_detection_model.pth"):
        violence_model.load_state_dict(torch.load("violence_detection_model.pth", map_location=device))
        violence_model.eval()
    else:
        print("Error: violence_detection_model.pth not found!")
else:
    print("Warning: ViolenceDetectionModel not found in training1.py")


def build_crowd_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax")  # Binary classification (Crowd / No Crowd)
    ])
    return model

crowd_model = build_crowd_model()
if os.path.exists("crowd_detection.h5"):
    crowd_model.load_weights("crowd_detection.h5")
else:
    print("Error: crowd_detection.h5 not found!")

def detect_safety(video_source):
    if video_source.startswith("rtsp") or video_source.startswith("http"):
        cap = cv2.VideoCapture(video_source)  # Read from CCTV
    else:
        cap = cv2.VideoCapture(video_source)  # Read from uploaded file

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return "Error: No frames found in the video."


    encrypted_frame = cipher.encrypt(frames[0].tobytes())


    decrypted_frame = np.frombuffer(cipher.decrypt(encrypted_frame), dtype=np.uint8).reshape(frames[0].shape)

    results = []


    if gender_model:
        frame = cv2.resize(decrypted_frame, (128, 128))
        frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        gender_pred = torch.argmax(gender_model(frame_tensor)).item()
        gender_result = "Female" if gender_pred == 1 else "Male"
        results.append(f"Gender: {gender_result}")


    if violence_model:
        violence_pred = torch.argmax(violence_model(frame_tensor)).item()
        violence_result = "Violence Detected" if violence_pred == 1 else "No Violence"
        results.append(f"Violence: {violence_result}")


    if crowd_model:
        frame = cv2.resize(decrypted_frame, (224, 224))
        frame_tensor = np.expand_dims(frame, axis=0) / 255.0  # Normalize for TensorFlow
        crowd_pred = np.argmax(crowd_model.predict(frame_tensor))
        crowd_result = "Crowd Present" if crowd_pred == 1 else "No Crowd"
        results.append(f"Crowd Detection: {crowd_result}")

    alert_message = "Safe Environment"
    if "Female" in results and ("Violence Detected" in results or "Crowd Present" in results):
        alert_message = "Emergency! Threat Detected!"

    return "\n".join(results) + f"\nAlert: {alert_message}"

iface = gr.Interface(
    fn=detect_safety,
    inputs=gr.Textbox(label="CCTV URL (RTSP/HTTP) or Upload Video"),
    outputs="text",
    title=" Women's Safety AI System",
    description="Enter a CCTV stream URL or upload a video for real-time safety analysis."
)

iface.launch(share=True)


Overwriting app.py


In [45]:
!python app.py

2025-03-21 15:49:53.342657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742572193.363033  130218 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742572193.369403  130218 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:49:57.779701: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1742572197.779882  130218 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12798 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


In [46]:
%%writefile app.py
import gradio as gr
import torch
import cv2
import numpy as np
import sys
import os
import tensorflow as tf
from cryptography.fernet import Fernet  # Encryption Library

# Ensure training1.py and training2.py are in the correct path
sys.path.append("/content/")

try:
    import training1
    import training2
except ModuleNotFoundError as e:
    print("Error: training1.py or training2.py not found. Make sure they are uploaded in /content/")
    raise e

# Generate Encryption Key (You can store it securely instead)
key = Fernet.generate_key()
cipher = Fernet(key)

# Load Available Models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load PyTorch Models (Gender & Violence Detection)
gender_model = None
violence_model = None
if hasattr(training1, "GenderClassificationModel"):
    gender_model = training1.GenderClassificationModel().to(device)
    if os.path.exists("gender_classification.pth"):
        gender_model.load_state_dict(torch.load("gender_classification.pth", map_location=device))
        gender_model.eval()
    else:
        print("Error: gender_classification.pth not found!")
else:
    print("Warning: GenderClassificationModel not found in training1.py")

if hasattr(training1, "ViolenceDetectionModel"):
    violence_model = training1.ViolenceDetectionModel().to(device)
    if os.path.exists("violence_detection_model.pth"):
        violence_model.load_state_dict(torch.load("violence_detection_model.pth", map_location=device))
        violence_model.eval()
    else:
        print("Error: violence_detection_model.pth not found!")
else:
    print("Warning: ViolenceDetectionModel not found in training1.py")

# Define Crowd Detection Model Before Loading Weights
def build_crowd_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax")  # Binary classification (Crowd / No Crowd)
    ])
    return model

crowd_model = build_crowd_model()
if os.path.exists("crowd_detection.h5"):
    crowd_model.load_weights("crowd_detection.h5")
else:
    print("Error: crowd_detection.h5 not found!")

# Function to process video frames (from CCTV or uploaded video)
def detect_safety(video_input, is_url):
    if is_url:
        cap = cv2.VideoCapture(video_input)  # Read from CCTV URL
    else:
        cap = cv2.VideoCapture(video_input)  # Read from uploaded file

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return "Error: No frames found in the video."

    # Encrypt the first frame before processing
    encrypted_frame = cipher.encrypt(frames[0].tobytes())

    # Decrypt for AI processing
    decrypted_frame = np.frombuffer(cipher.decrypt(encrypted_frame), dtype=np.uint8).reshape(frames[0].shape)

    results = []

    # Run Gender Classification (PyTorch)
    if gender_model:
        frame = cv2.resize(decrypted_frame, (128, 128))
        frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        gender_pred = torch.argmax(gender_model(frame_tensor)).item()
        gender_result = "Female" if gender_pred == 1 else "Male"
        results.append(f"Gender: {gender_result}")

    # Run Violence Detection (PyTorch)
    if violence_model:
        violence_pred = torch.argmax(violence_model(frame_tensor)).item()
        violence_result = "Violence Detected" if violence_pred == 1 else "No Violence"
        results.append(f"Violence: {violence_result}")

    # Run Crowd Detection (TensorFlow)
    if crowd_model:
        frame = cv2.resize(decrypted_frame, (224, 224))
        frame_tensor = np.expand_dims(frame, axis=0) / 255.0  # Normalize for TensorFlow
        crowd_pred = np.argmax(crowd_model.predict(frame_tensor))
        crowd_result = "Crowd Present" if crowd_pred == 1 else "No Crowd"
        results.append(f"Crowd Detection: {crowd_result}")

    # Decision Logic
    alert_message = "Safe Environment"
    if "Female" in results and ("Violence Detected" in results or "Crowd Present" in results):
        alert_message = "Emergency! Threat Detected!"

    return "\n".join(results) + f"\nAlert: {alert_message}"

# Gradio Interface with CCTV & Upload Support
with gr.Blocks() as iface:
    gr.Markdown(
        """
        <h1 style="text-align: center; font-size: 2.5em; color: #333;">संरक्षक</h1>
        <h2 style="text-align: center; font-size: 1.8em; color: #666;">Together, Let's Redefine Safety. For Women. For Everyone.</h2>
        <p style="text-align: center; font-size: 1.2em; color: #888;">
        "A nation’s progress is measured by the status and safety of its women." <br> — Jawaharlal Nehru
        </p>
        """,
        unsafe_allow_html=True
    )

    with gr.Row():
        with gr.Column():
            gr.Markdown("## CCTV Live Streaming")
            cctv_url = gr.Textbox(label="Enter CCTV Stream URL (RTSP/HTTP)")
            analyze_cctv = gr.Button("Analyze CCTV")
            cctv_output = gr.Textbox(label="CCTV Analysis Results")
            analyze_cctv.click(fn=detect_safety, inputs=[cctv_url, gr.Boolean(value=True)], outputs=cctv_output)

        with gr.Column():
            gr.Markdown("## Upload Video")
            uploaded_video = gr.Video(label="Upload a Video File")
            analyze_video = gr.Button("Analyze Video")
            video_output = gr.Textbox(label="Video Analysis Results")
            analyze_video.click(fn=detect_safety, inputs=[uploaded_video, gr.Boolean(value=False)], outputs=video_output)

    gr.Markdown(
        """
        <br><br>
        <p style="text-align: center; font-size: 1.2em; color: #777;">
        Encrypted CCTV & Video Processing for Enhanced Security
        </p>
        """,
        unsafe_allow_html=True
    )

iface.launch(share=True)


Overwriting app.py


In [47]:
!python app.py

2025-03-21 15:57:42.447565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742572662.468759  132263 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742572662.475350  132263 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:57:47.434805: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1742572667.435022  132263 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12798 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


In [48]:
%%writefile app.py
import gradio as gr
import torch
import cv2
import numpy as np
import sys
import os
import tensorflow as tf
from cryptography.fernet import Fernet  # Encryption Library

# Ensure training1.py and training2.py are in the correct path
sys.path.append("/content/")

try:
    import training1
    import training2
except ModuleNotFoundError as e:
    print("Error: training1.py or training2.py not found. Make sure they are uploaded in /content/")
    raise e

# Generate Encryption Key (You can store it securely instead)
key = Fernet.generate_key()
cipher = Fernet(key)

# Load Available Models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load PyTorch Models (Gender & Violence Detection)
gender_model = None
violence_model = None
if hasattr(training1, "GenderClassificationModel"):
    gender_model = training1.GenderClassificationModel().to(device)
    if os.path.exists("gender_classification.pth"):
        gender_model.load_state_dict(torch.load("gender_classification.pth", map_location=device))
        gender_model.eval()
    else:
        print("Error: gender_classification.pth not found!")
else:
    print("Warning: GenderClassificationModel not found in training1.py")

if hasattr(training1, "ViolenceDetectionModel"):
    violence_model = training1.ViolenceDetectionModel().to(device)
    if os.path.exists("violence_detection_model.pth"):
        violence_model.load_state_dict(torch.load("violence_detection_model.pth", map_location=device))
        violence_model.eval()
    else:
        print("Error: violence_detection_model.pth not found!")
else:
    print("Warning: ViolenceDetectionModel not found in training1.py")

# Define Crowd Detection Model Before Loading Weights
def build_crowd_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax")  # Binary classification (Crowd / No Crowd)
    ])
    return model

crowd_model = build_crowd_model()
if os.path.exists("crowd_detection.h5"):
    crowd_model.load_weights("crowd_detection.h5")
else:
    print("Error: crowd_detection.h5 not found!")

# Function to process video frames (from CCTV or uploaded video)
def detect_safety(video_input, is_url):
    if is_url:
        cap = cv2.VideoCapture(video_input)  # Read from CCTV URL
    else:
        cap = cv2.VideoCapture(video_input)  # Read from uploaded file

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return "Error: No frames found in the video."

    # Encrypt the first frame before processing
    encrypted_frame = cipher.encrypt(frames[0].tobytes())

    # Decrypt for AI processing
    decrypted_frame = np.frombuffer(cipher.decrypt(encrypted_frame), dtype=np.uint8).reshape(frames[0].shape)

    results = []

    # Run Gender Classification (PyTorch)
    if gender_model:
        frame = cv2.resize(decrypted_frame, (128, 128))
        frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        gender_pred = torch.argmax(gender_model(frame_tensor)).item()
        gender_result = "Female" if gender_pred == 1 else "Male"
        results.append(f"Gender: {gender_result}")

    # Run Violence Detection (PyTorch)
    if violence_model:
        violence_pred = torch.argmax(violence_model(frame_tensor)).item()
        violence_result = "Violence Detected" if violence_pred == 1 else "No Violence"
        results.append(f"Violence: {violence_result}")

    # Run Crowd Detection (TensorFlow)
    if crowd_model:
        frame = cv2.resize(decrypted_frame, (224, 224))
        frame_tensor = np.expand_dims(frame, axis=0) / 255.0  # Normalize for TensorFlow
        crowd_pred = np.argmax(crowd_model.predict(frame_tensor))
        crowd_result = "Crowd Present" if crowd_pred == 1 else "No Crowd"
        results.append(f"Crowd Detection: {crowd_result}")

    # Decision Logic
    alert_message = "Safe Environment"
    if "Female" in results and ("Violence Detected" in results or "Crowd Present" in results):
        alert_message = "Emergency! Threat Detected!"

    return "\n".join(results) + f"\nAlert: {alert_message}"

# Gradio Interface with CCTV & Upload Support
with gr.Blocks() as iface:
    gr.HTML(
        """
        <h1 style="text-align: center; font-size: 2.5em; color: #333;">संरक्षक</h1>
        <h2 style="text-align: center; font-size: 1.8em; color: #666;">Together, Let's Redefine Safety. For Women. For Everyone.</h2>
        <p style="text-align: center; font-size: 1.2em; color: #888;">
        "A nation’s progress is measured by the status and safety of its women." <br> — Jawaharlal Nehru
        </p>
        """
    )

    with gr.Row():
        with gr.Column():
            gr.Markdown("## CCTV Live Streaming")
            cctv_url = gr.Textbox(label="Enter CCTV Stream URL (RTSP/HTTP)")
            analyze_cctv = gr.Button("Analyze CCTV")
            cctv_output = gr.Textbox(label="CCTV Analysis Results")
            analyze_cctv.click(fn=detect_safety, inputs=[cctv_url, gr.Boolean(value=True)], outputs=cctv_output)

        with gr.Column():
            gr.Markdown("## Upload Video")
            uploaded_video = gr.Video(label="Upload a Video File")
            analyze_video = gr.Button("Analyze Video")
            video_output = gr.Textbox(label="Video Analysis Results")
            analyze_video.click(fn=detect_safety, inputs=[uploaded_video, gr.Boolean(value=False)], outputs=video_output)

    gr.HTML(
        """
        <br><br>
        <p style="text-align: center; font-size: 1.2em; color: #777;">
        Encrypted CCTV & Video Processing for Enhanced Security
        </p>
        """
    )

iface.launch(share=True)


Overwriting app.py


In [49]:
!python app.py

2025-03-21 15:59:25.689803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742572765.710556  132737 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742572765.717035  132737 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:59:30.160179: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1742572770.160352  132737 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12798 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


In [50]:
%%writefile app.py
import gradio as gr
import torch
import cv2
import numpy as np
import sys
import os
import tensorflow as tf
from cryptography.fernet import Fernet  # Encryption Library

# Ensure training1.py and training2.py are in the correct path
sys.path.append("/content/")

try:
    import training1
    import training2
    from importlib import reload
    training1 = reload(training1)
except ModuleNotFoundError as e:
    print("Error: training1.py or training2.py not found. Make sure they are uploaded in /content/")
    raise e

# Generate Encryption Key (You can store it securely instead)
key = Fernet.generate_key()
cipher = Fernet(key)

# Load Available Models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load PyTorch Models (Gender & Violence Detection)
gender_model = None
violence_model = None
if hasattr(training1, "GenderClassificationModel"):
    gender_model = training1.GenderClassificationModel().to(device)
    if os.path.exists("gender_classification.pth"):
        gender_model.load_state_dict(torch.load("gender_classification.pth", map_location=device))
        gender_model.eval()
    else:
        print("Error: gender_classification.pth not found!")
else:
    print("Warning: GenderClassificationModel not found in training1.py")

if hasattr(training1, "ViolenceDetectionModel"):
    violence_model = training1.ViolenceDetectionModel().to(device)
    if os.path.exists("violence_detection_model.pth"):
        violence_model.load_state_dict(torch.load("violence_detection_model.pth", map_location=device))
        violence_model.eval()
    else:
        print("Error: violence_detection_model.pth not found!")
else:
    print("Warning: ViolenceDetectionModel not found in training1.py")

# Define Crowd Detection Model Before Loading Weights
def build_crowd_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(224, 224, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax")  # Binary classification (Crowd / No Crowd)
    ])
    return model

crowd_model = build_crowd_model()
if os.path.exists("crowd_detection.h5"):
    crowd_model.load_weights("crowd_detection.h5")
else:
    print("Error: crowd_detection.h5 not found!")

# Function to process video frames (from CCTV or uploaded video)
def detect_safety(video_input, is_url):
    if is_url:
        cap = cv2.VideoCapture(video_input)  # Read from CCTV URL
    else:
        cap = cv2.VideoCapture(video_input)  # Read from uploaded file

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return "Error: No frames found in the video."

    # Encrypt the first frame before processing
    encrypted_frame = cipher.encrypt(frames[0].tobytes())

    # Decrypt for AI processing
    decrypted_frame = np.frombuffer(cipher.decrypt(encrypted_frame), dtype=np.uint8).reshape(frames[0].shape)

    results = []

    # Run Gender Classification (PyTorch)
    if gender_model:
        frame = cv2.resize(decrypted_frame, (128, 128))
        frame_tensor = torch.tensor(frame, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        gender_pred = torch.argmax(gender_model(frame_tensor)).item()
        gender_result = "Female" if gender_pred == 1 else "Male"
        results.append(f"Gender: {gender_result}")

    # Run Violence Detection (PyTorch)
    if violence_model:
        violence_pred = torch.argmax(violence_model(frame_tensor)).item()
        violence_result = "Violence Detected" if violence_pred == 1 else "No Violence"
        results.append(f"Violence: {violence_result}")

    # Run Crowd Detection (TensorFlow)
    if crowd_model:
        frame = cv2.resize(decrypted_frame, (224, 224))
        frame_tensor = np.expand_dims(frame, axis=0) / 255.0  # Normalize for TensorFlow
        crowd_pred = np.argmax(crowd_model.predict(frame_tensor))
        crowd_result = "Crowd Present" if crowd_pred == 1 else "No Crowd"
        results.append(f"Crowd Detection: {crowd_result}")

    # Decision Logic
    alert_message = "Safe Environment"
    if "Female" in results and ("Violence Detected" in results or "Crowd Present" in results):
        alert_message = "Emergency! Threat Detected!"

    return "\n".join(results) + f"\nAlert: {alert_message}"

# Gradio Interface with CCTV & Upload Support
with gr.Blocks() as iface:
    gr.HTML(
        """
        <h1 style="text-align: center; font-size: 2.5em; color: #333;">संरक्षक</h1>
        <h2 style="text-align: center; font-size: 1.8em; color: #666;">Together, Let's Redefine Safety. For Women. For Everyone.</h2>
        <p style="text-align: center; font-size: 1.2em; color: #888;">
        "A nation’s progress is measured by the status and safety of its women." <br> — Jawaharlal Nehru
        </p>
        """
    )

    with gr.Row():
        with gr.Column():
            gr.Markdown("## CCTV Live Streaming")
            cctv_url = gr.Textbox(label="Enter CCTV Stream URL (RTSP/HTTP)")
            analyze_cctv = gr.Button("Analyze CCTV")
            cctv_output = gr.Textbox(label="CCTV Analysis Results")
            analyze_cctv.click(fn=detect_safety, inputs=[cctv_url, gr.Checkbox(value=True, visible=False)], outputs=cctv_output)

        with gr.Column():
            gr.Markdown("## Upload Video")
            uploaded_video = gr.Video(label="Upload a Video File")
            analyze_video = gr.Button("Analyze Video")
            video_output = gr.Textbox(label="Video Analysis Results")
            analyze_video.click(fn=detect_safety, inputs=[uploaded_video, gr.Checkbox(value=False, visible=False)], outputs=video_output)

iface.launch(share=True)


Overwriting app.py


In [ ]:
!python app.py

2025-03-21 16:03:43.397831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742573023.418682  133883 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742573023.425091  133883 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 16:03:48.770361: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1742573028.770577  133883 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12798 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
